# Artificial neural network

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.8.0'

## Part1 Data preprocessing

### importing the dataset

In [3]:
dataset = pd.read_csv("Churn_Modelling.csv")

In [5]:
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [6]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [7]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

In [8]:
# label encoding gender column

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [9]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [11]:
# One-hot encoding the Geography column

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[("encoder", OneHotEncoder(), [1])], remainder="passthrough")
X = np.array(ct.fit_transform(X))

In [12]:
print(X)

[[1.0 0.0 1.0 ... 1 1 101348.88]
 [1.0 0.0 0.0 ... 0 1 112542.58]
 [1.0 0.0 1.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 1.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 1.0 ... 1 0 38190.78]]


In [14]:
# Splitting the dataset into the Training set and Test set

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)    

In [15]:
# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part2 - Building the ANN

In [16]:
# initializing the ANN
ann = tf.keras.models.Sequential()

In [17]:
# Adding the input layer and the first hidden layer

ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [18]:
# Adding the second hidden layer

ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [19]:
# Adding the output layer

ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part3 - Training the ANN

In [20]:
ann.compile(optimizer="adam", loss="binary_crossentropy" , metrics=['accuracy'])

In [21]:
# Training the ANN on the Training set

ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 2s 2ms/step - loss: 0.5247 - accuracy: 0.7810
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4587 - accuracy: 0.7995
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4420 - accuracy: 0.8052
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4351 - accuracy: 0.8058
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4298 - accuracy: 0.8100
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4249 - accuracy: 0.8117
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4187 - accuracy: 0.8155
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4102 - accuracy: 0.8226
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3984 - accuracy: 0.8295
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3840 - accura

## Part4 - Making the predictions and evaluating the model

In [25]:
# Predicting the result of a single observation

print(ann.predict(sc.transform([[1, 0, 1, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


In [26]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [28]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1509   86]
 [ 197  208]]


0.8585